In [8]:
# import drf

# drf.hello(1, 2)

4

In [4]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro

import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2 import robjects
from rpy2.robjects.vectors import IntVector
rpy2.robjects.NameClassMap = rpy2.robjects.conversion.NameClassMap
from rpy2_Matrix import Matrix
from rpy2_Matrix import conversion

# Choosing a CRAN Mirror
#import rpy2.robjects.packages as rpackages
#utils = rpackages.importr('utils')
#utils.chooseCRANmirror(ind=1)

# Installing required packages
#from rpy2.robjects.vectors import StrVector
#packages = ('drf')
#utils.install_packages(StrVector(packages))

# Import packages
from rpy2.robjects.packages import importr
base, drf_r_package = importr('base'), importr('drf')

# Import Functions
#fit, predict = drf.drf, drf.predict_drf

AttributeError: module 'rpy2.robjects' has no attribute 'rs4map_context'

In [2]:
def convert_to_R(X):
    if type(X) == np.ndarray:
        if len(X.shape) == 1: 
            X = X.reshape(1, X.shape[0])
        nr, nc = X.shape
        return ro.r.matrix(X, nrow=nr, ncol=nc)

    if type(X) == pd.core.frame.DataFrame:
        return ro.conversion.py2rpy(X)

class drf:
    def __init__(self, **fit_params):
        self.fit_params = fit_params 
    
    class predict_output: pass
    
    def fit(self, X, Y):
        self.X_train = X
        self.Y_train = Y
        
        X_r = convert_to_R(X)
        Y_r = convert_to_R(Y)
        
        self.r_fit_object = drf_r_package.drf(X_r, Y_r, **self.fit_params)

    def myFun(**kwargs):  
        for key, value in kwargs.items(): 
            print ("%s == %s" %(key, value)) 

    def predict(self, newdata, **predict_params):
        newdata_r = convert_to_R(newdata)
        r_output = drf_r_package.predict_drf(self.r_fit_object, newdata_r, **predict_params)

        ret = self.predict_output()
        for i in range(len(r_output)):
            value = r_output[i]
            if r_output.names[i] == 'weights':
                value = base.as_matrix(r_output[i])
            setattr(ret, r_output.names[i], value)
        return ret

In [4]:
import numpy as np
import pandas as pd

n=1000
p=10
d=2
X = np.random.normal(0, 1, size=(n, p))
Y = np.random.normal(0, 1, size=(n, d))
Y[:,0] = Y[:,0] + X[:,0]
Y[:,1] = Y[:,1] * X[:,1]

#X = pd.DataFrame(X)
#Y = pd.DataFrame(Y)

In [ ]:
regr = drf(mtry=3, bandwidth=1.1, min_node_size=10, num_trees=100, num_features=10, splitting_rule="CART")
regr.fit(X, Y)
out = regr.predict(newdata=X[1,:], functional='mean', transformation=lambda y : y)#, quantiles=np.array([0.1, 0.5, 0.9]))
#fix quantiles parameter to be of any type
#reformat quantiles output